In [24]:
import tensorflow
from tensorflow import keras
import numpy as np
import os
from glob import glob
from glob import iglob
from PIL import Image

In [25]:
# Image Resizing Method
def resize_images(img_path):
	images=glob(img_path + "/*.png")  
 
	print(len(images), " images to be resized.")
 
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "PNG")
  
	print(len(images), " images resized.")

# Data Loading Method
def load_data(img_path, number_of_data=100):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=4
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in iglob(img_path+'/scissors/*.png'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=0
        idx=idx+1

    for file in iglob(img_path+'/rock/*.png'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1
        idx=idx+1  

    for file in iglob(img_path+'/paper/*.png'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=2
        idx=idx+1
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [27]:
# Resize the Data Images : Train
image_dir_path = "dataset/train/scissors"
resize_images(image_dir_path)
image_dir_path =  "dataset/train/rock"
resize_images(image_dir_path)
image_dir_path = "dataset/train/paper"
resize_images(image_dir_path)

# Resize the Data Images : Test
image_dir_path = "dataset/test/scissors"
resize_images(image_dir_path)
image_dir_path =  "dataset/test/rock"
resize_images(image_dir_path)
image_dir_path = "dataset/test/paper"
resize_images(image_dir_path)


840  images to be resized.
840  images resized.
840  images to be resized.
840  images resized.
840  images to be resized.
840  images resized.
124  images to be resized.
124  images resized.
124  images to be resized.
124  images resized.
124  images to be resized.
124  images resized.


In [35]:
# Load the Dataset : Train
image_dir_path = "dataset/train"
x_train, y_train = load_data(image_dir_path, number_of_data=2520)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

# Load the Dataset : Test
image_dir_path = "dataset/test"
x_test, y_test = load_data(image_dir_path, number_of_data=372)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화
print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))


학습데이터(x_train)의 이미지 개수는 2520 입니다.
x_train shape: (2520, 28, 28, 4)
y_train shape: (2520,)
학습데이터(x_train)의 이미지 개수는 372 입니다.
x_test shape: (372, 28, 28, 4)
y_test shape: (372,)


In [36]:

# 하이퍼 파라미터
# kernel_regularizer = keras.regularizers.L1(0.001)
kernel_regularizer = None

# 모델 설계
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=(28,28,4)))

model.add(keras.layers.Conv2D(16, (3,3), kernel_regularizer=kernel_regularizer))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(32, (3,3), kernel_regularizer=kernel_regularizer))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64, (3,3), kernel_regularizer=kernel_regularizer))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(16, activation='softmax'))

# 모델 개요
model.summary()

# 모델 컴파일
optimizer = 'adam'
loss = 'sparse_categorical_crossentropy',
metrics = ['categorical_accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 8)         296       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 16)        1168      
_________________________________________________________________
dropout_12 (Dropout)         (None, 24, 24, 16)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
activation_12 (Activation)   (None, 24, 24, 16)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 32)       

In [ ]:
# 콜백 지정
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]

# 모델 학습
model.fit(x_train_norm, y_train, validation_split=0.3, epochs=1000)

# 모델 평가
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

Epoch 1/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0020 - categorical_accuracy: 0.4762 - val_loss: 0.7562 - val_categorical_accuracy: 0.0053
Epoch 2/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0021 - categorical_accuracy: 0.4768 - val_loss: 0.8227 - val_categorical_accuracy: 0.0053
Epoch 3/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0020 - categorical_accuracy: 0.4762 - val_loss: 0.3362 - val_categorical_accuracy: 0.0635
Epoch 4/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0183 - categorical_accuracy: 0.4785 - val_loss: 9.9391e-04 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
56/56 [==============================] - 0s 5ms/step - loss: 0.0035 - categorical_accuracy: 0.4768 - val_loss: 0.1924 - val_categorical_accuracy: 0.0159
Epoch 6/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0016 - categorical_accuracy: 0.4762 - val_loss: 0.1703 - val_categorical_accuracy: 0.0

56/56 [==============================] - 0s 6ms/step - loss: 2.8539e-04 - categorical_accuracy: 0.4762 - val_loss: 0.3746 - val_categorical_accuracy: 0.0410
Epoch 50/1000
56/56 [==============================] - 0s 6ms/step - loss: 2.5115e-04 - categorical_accuracy: 0.4762 - val_loss: 0.2390 - val_categorical_accuracy: 0.0410
Epoch 51/1000
56/56 [==============================] - 0s 5ms/step - loss: 2.1841e-04 - categorical_accuracy: 0.4762 - val_loss: 0.2334 - val_categorical_accuracy: 0.0251
Epoch 52/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.5667e-04 - categorical_accuracy: 0.4762 - val_loss: 0.3160 - val_categorical_accuracy: 0.0198
Epoch 53/1000
56/56 [==============================] - 0s 6ms/step - loss: 1.0489e-04 - categorical_accuracy: 0.4762 - val_loss: 0.3258 - val_categorical_accuracy: 0.0198
Epoch 54/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.3676e-04 - categorical_accuracy: 0.4762 - val_loss: 0.4642 - val_categorical_accur

Epoch 97/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0024 - categorical_accuracy: 0.4762 - val_loss: 0.2438 - val_categorical_accuracy: 0.0000e+00
Epoch 98/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0102 - categorical_accuracy: 0.4768 - val_loss: 5.8391 - val_categorical_accuracy: 0.0159
Epoch 99/1000
56/56 [==============================] - 0s 6ms/step - loss: 0.0027 - categorical_accuracy: 0.4762 - val_loss: 0.0358 - val_categorical_accuracy: 0.0013
Epoch 100/1000
56/56 [==============================] - 0s 6ms/step - loss: 4.5124e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0299 - val_categorical_accuracy: 0.0000e+00
Epoch 101/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.6525e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0692 - val_categorical_accuracy: 0.0026
Epoch 102/1000
56/56 [==============================] - 0s 6ms/step - loss: 1.6130e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0941 - val_cate

56/56 [==============================] - 0s 6ms/step - loss: 9.0502e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0348 - val_categorical_accuracy: 0.0159
Epoch 145/1000
56/56 [==============================] - 0s 5ms/step - loss: 7.3563e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0353 - val_categorical_accuracy: 0.0159
Epoch 146/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.9574e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0291 - val_categorical_accuracy: 0.0066
Epoch 147/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.6321e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0094 - val_categorical_accuracy: 0.0013
Epoch 148/1000
56/56 [==============================] - 0s 5ms/step - loss: 3.4082e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0157 - val_categorical_accuracy: 0.0053
Epoch 149/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.4461e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0196 - val_categorical_

Epoch 192/1000
56/56 [==============================] - 0s 6ms/step - loss: 1.6056e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0612 - val_categorical_accuracy: 0.0198
Epoch 193/1000
56/56 [==============================] - 0s 6ms/step - loss: 2.5516e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0560 - val_categorical_accuracy: 0.0198
Epoch 194/1000
56/56 [==============================] - 0s 6ms/step - loss: 1.7344e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0591 - val_categorical_accuracy: 0.0198
Epoch 195/1000
56/56 [==============================] - 0s 6ms/step - loss: 5.7594e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0578 - val_categorical_accuracy: 0.0198
Epoch 196/1000
56/56 [==============================] - 0s 5ms/step - loss: 3.9786e-05 - categorical_accuracy: 0.4762 - val_loss: 0.1310 - val_categorical_accuracy: 0.0172
Epoch 197/1000
56/56 [==============================] - 0s 5ms/step - loss: 2.8822e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0542 - v

56/56 [==============================] - 0s 5ms/step - loss: 0.0064 - categorical_accuracy: 0.4756 - val_loss: 0.0394 - val_categorical_accuracy: 0.0066
Epoch 240/1000
56/56 [==============================] - 0s 6ms/step - loss: 4.0092e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0470 - val_categorical_accuracy: 0.0040
Epoch 241/1000
56/56 [==============================] - 0s 5ms/step - loss: 2.4272e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0406 - val_categorical_accuracy: 0.0026
Epoch 242/1000
56/56 [==============================] - 0s 5ms/step - loss: 2.2878e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0572 - val_categorical_accuracy: 0.0026
Epoch 243/1000
56/56 [==============================] - 0s 6ms/step - loss: 9.8907e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0781 - val_categorical_accuracy: 0.0040
Epoch 244/1000
56/56 [==============================] - 0s 6ms/step - loss: 1.0802e-04 - categorical_accuracy: 0.4762 - val_loss: 0.0785 - val_categorical_accu

56/56 [==============================] - 0s 5ms/step - loss: 1.7940e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0866 - val_categorical_accuracy: 0.0000e+00
Epoch 287/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.0152e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0866 - val_categorical_accuracy: 0.0000e+00
Epoch 288/1000
56/56 [==============================] - 0s 5ms/step - loss: 9.7760e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0906 - val_categorical_accuracy: 0.0000e+00
Epoch 289/1000
56/56 [==============================] - 0s 6ms/step - loss: 9.3272e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0906 - val_categorical_accuracy: 0.0000e+00
Epoch 290/1000
56/56 [==============================] - 0s 6ms/step - loss: 9.0290e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0861 - val_categorical_accuracy: 0.0000e+00
Epoch 291/1000
56/56 [==============================] - 0s 6ms/step - loss: 1.4090e-05 - categorical_accuracy: 0.4762 - val_loss: 0.080

Epoch 333/1000
56/56 [==============================] - 0s 6ms/step - loss: 9.3343e-06 - categorical_accuracy: 0.4762 - val_loss: 0.1414 - val_categorical_accuracy: 0.0053
Epoch 334/1000
56/56 [==============================] - 0s 6ms/step - loss: 5.2727e-06 - categorical_accuracy: 0.4762 - val_loss: 0.1416 - val_categorical_accuracy: 0.0040
Epoch 335/1000
56/56 [==============================] - 0s 6ms/step - loss: 7.4338e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0659 - val_categorical_accuracy: 0.0000e+00
Epoch 336/1000
56/56 [==============================] - 0s 5ms/step - loss: 3.5727e-05 - categorical_accuracy: 0.4762 - val_loss: 0.1453 - val_categorical_accuracy: 0.0000e+00
Epoch 337/1000
56/56 [==============================] - 0s 5ms/step - loss: 1.8470e-05 - categorical_accuracy: 0.4762 - val_loss: 0.1440 - val_categorical_accuracy: 0.0000e+00
Epoch 338/1000
56/56 [==============================] - 0s 5ms/step - loss: 6.0357e-06 - categorical_accuracy: 0.4762 - val_loss

56/56 [==============================] - 0s 6ms/step - loss: 3.7976e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0609 - val_categorical_accuracy: 0.0013
Epoch 381/1000
56/56 [==============================] - 0s 5ms/step - loss: 9.9885e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0679 - val_categorical_accuracy: 0.0000e+00
Epoch 382/1000
56/56 [==============================] - 0s 6ms/step - loss: 2.7671e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0633 - val_categorical_accuracy: 0.0000e+00
Epoch 383/1000
56/56 [==============================] - 0s 6ms/step - loss: 4.3934e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0611 - val_categorical_accuracy: 0.0013
Epoch 384/1000
56/56 [==============================] - 0s 6ms/step - loss: 4.3980e-06 - categorical_accuracy: 0.4762 - val_loss: 0.0558 - val_categorical_accuracy: 0.0013
Epoch 385/1000
56/56 [==============================] - 0s 6ms/step - loss: 6.1282e-05 - categorical_accuracy: 0.4762 - val_loss: 0.0395 - val_cate